# AI Agents with Generative AI Hub

Everyone is talking about AI Agents and how they are going to make LLMs more powerful. AI Agents are basically systems that have a set of tools that they can use and the LLM is the agents brain to decide which tool to use when. Some systems are more agentic than others, depending on the amount of autonomy and decision-making power they possess. 

If you want to know more about AI Agents and how to build one from scratch, check out this [Devtoberfest session](https://community.sap.com/t5/devtoberfest/getting-started-with-agents-using-sap-generative-ai-hub/ev-p/13865119).

👉 Before you start you will need to add the constant `LLM_DEPLOYMENT_ID` to [variables.py](variables.py). You can use the `gpt-4.1-mini` model again that we deployed earlier. You find the deployment ID in SAP AI Launchpad.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message=".*tracemalloc.*")

import init_env
import variables

init_env.set_environment_variables()

### Import the packages you want to use

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langchain import hub
from langgraph.prebuilt import create_react_agent

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings


We will give the agent different tools to use. The first tool will be [access to Wikipedia](https://python.langchain.com/docs/integrations/tools/wikipedia/). This way instead of answering from it's training data, the model will check on Wikipedia articles first.

In [ ]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Let's check if it works
print(wikipedia.run("Tbilisi"))

# Assign wikipedia to the set of tools
tools = [wikipedia]

In [ ]:
# For agents the initial prompt including all the instructions is very important. 
# LangChain already has a good set of prompts that we can reuse here.
prompt = hub.pull("hwchase17/react")
print(prompt.template)

In [ ]:
import importlib
variables = importlib.reload(variables)

# Create a chat model instance which will act as the brain of the agent
llm = ChatOpenAI(deployment_id=variables.LLM_DEPLOYMENT_ID)

# Create an agent with the llm, tools and prompt
agent = create_react_agent(llm, tools)

# Let's ask try the agent and ask about the city
result = agent.invoke({
            "messages": [
                ("user", f"{prompt}\n\nTell me about Tbilisi!")
            ]
        })

## Printing the result of the agent

In [ ]:
# Printing the results
from IPython.display import Markdown, display

# Display as formatted markdown
final_response = result['messages'][-1].content
display(Markdown(f"## Agent Response\n\n{final_response}"))

# Or show the reasoning process
print("=== Agent Thinking Process ===")
for msg in result['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"🔧 Agent decided to use tool: {msg.tool_calls[0]['name']}")
    elif msg.__class__.__name__ == 'ToolMessage':
        print(f"📋 Tool returned data (length: {len(msg.content)} chars)")
    elif msg.__class__.__name__ == 'AIMessage' and msg.content:
        print(f"🤖 Final response ready")

## Give the agent access to our HANA vector store
Now the agent can use Wikipedia but wouldn't it be nice if the agent could pick between resources and decide when to use what?

In [ ]:
# LangChain & HANA Vector Engine - legacy integration: https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.hanavector.HanaDB.html
#from langchain_community.vectorstores.hanavector import HanaDB

# SAP HANA integration with LangChain: https://pypi.org/project/langchain-hana 
from langchain_hana import HanaDB
from langchain.tools.retriever import create_retriever_tool

# connect to HANA instance
connection = init_env.connect_to_hana_db()
connection.isconnected()

# Reference which embedding model, DB connection and table to use
embeddings = OpenAIEmbeddings(deployment_id=variables.EMBEDDING_DEPLOYMENT_ID)
db = HanaDB(
    embedding=embeddings, connection=connection, table_name=variables.EMBEDDING_TABLE
)

# Create a retriever instance of the vector store
retriever = db.as_retriever(search_kwargs={"k": 2})

# We need to add a description to the retriever so that the llm knows when to use this tool.
retriever_tool = create_retriever_tool(
    retriever,
    "SAP-orchestration-service-docu",
    "Search for information SAP's orchestration service. For all inquiries specifically related to the Generative AI Hub and SAP's orchestration service, this tool must be used without exception!",
)

## Ask the Agent

You can ask the agent anything in general and it will try to find an entry from Wikipedia, unless you ask about SAP's orchestration service. Then it will get the information from SAP HANA vector store, where it holds the documentation, instead of Wikipedia.

In [ ]:
# Now let's add the retriever as a tool
tools = [wikipedia, retriever_tool]

# And create the agent again with the two tools, wikipedia and the HANA vector store (retriever)
# Create an agent with the llm, tools and prompt
agent = create_react_agent(llm, tools)

# First ask: What is Data Masking?
# Then ask: What is Data Masking in SAP's Generative AI Hub?
# Pay attention to the response! Do you see what is happening now?
result = agent.invoke({
            "messages": [
                ("user", f"{prompt}\n\nWhat is Data Masking in SAP's Generative AI Hub?")
            ]
        })

In [ ]:
# Display as formatted markdown
final_response = result['messages'][-1].content
display(Markdown(f"## Agent Response\n\n{final_response}"))

# Or show the reasoning process
print("=== Agent Thinking Process ===")
for msg in result['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"🔧 Agent decided to use tool: {msg.tool_calls[0]['name']}")
    elif msg.__class__.__name__ == 'ToolMessage':
        print(f"📋 Tool returned data (length: {len(msg.content)} chars)")
    elif msg.__class__.__name__ == 'AIMessage' and msg.content:
        print(f"🤖 Final response ready")